In [6]:
from operator import itemgetter

import opencc
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_together.llms import Together

load_dotenv()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_prompt = """
You are a science communicator specializing in astronomy. Your task is to elucidate the vastness of the universe to the general public, employing vivid size comparisons that are relatable in everyday life. For example, when describing a galaxy, you might liken it to a sea of stars, each potentially hosting its own worlds, akin to grains of sand on a beach. However, it's crucial to include actual data with numbers, such as distances in light-years, sizes in comparison to Earth or the Sun, and any pertinent scientific measurements. Your explanations should effectively bridge the gap between imaginative understanding and factual accuracy, rendering the marvels of the cosmos both accessible and fascinating to a broad audience.
"""

user_prompt = """
{question}
"""


def s2hk(content):
    converter = opencc.OpenCC("s2hk")
    return converter.convert(content)


def get_answer(question, system_prompt, user_prompt):
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template(user_prompt),
        ]
    )

    model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
    # model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)
    # model = AzureChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
    # model = Together(model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.7, max_tokens=4096)
    # model = Together(model="deepseek-ai/deepseek-coder-33b-instruct", temperature=0.7, max_tokens=4096)

    chain = (
        RunnablePassthrough.assign(chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"))
        | prompt
        | model
        | StrOutputParser()
        | RunnableLambda(s2hk)
    )

    with get_openai_callback() as callback:
        response = chain.invoke({"question": question})
        print(callback, end="\n\n")
    print(response)
    memory.save_context({"question": question}, {"response": response})


# memory.load_memory_variables({})

In [7]:
get_answer("Describe Sagittarius A* and TON 618.", system_prompt, user_prompt)